In [96]:
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy import optimize
import sympy as sm
from scipy.optimize import minimize 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import numpy as np
from scipy.optimize import minimize


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

We consider a Malthus economy where we apply fertility as the main factor of this model. 

The utility function of household depends on consumption $c_t$ and number of surviving children $n_t$ where $\beta \epsilon (0.1)$.  

$$u_t(c_t) = \beta\ln(c_t) + (1-\beta)\ln(n_t)$$

The budget constraint is given as:

$$c_t + \lambda n_t = y_t (1-\tau)$$

Where c is consumption, $\lambda$ is relative price on 1 child, $y_t$ is the average income and $\tau$ is income after tax. 

Income per capita is given by: 
$y_t \equiv (\frac{AX}{L_t})^{\alpha}$

We solve the problem by inserting the budget contraint in the utility function: 

$$\max_{u_t}=\beta log[y_t (1-\tau)-\lambda n_t]+(1-\beta) log (n_t)$$

We take FOC wrt. $n_t$
$$\frac{\partial u}{\partial n}=0: \beta \frac{-\lambda}{y_t (1-\tau)-\lambda n_t}+(1-\beta) \frac{1}{n_t}=0$$

Isolating $n_t$ yields:
$$n_t=\frac{1-\beta}{\lambda}y_t(1-\tau)$$

Where:
$$n_t=\frac{1-\beta}{\lambda}y_t(1-\tau) \equiv \eta y_t$$

And $$\eta \equiv\frac{1-\beta}{\lambda}(1-\tau)$$

Population in the next period is the number of surviving children, and the the part of the population that survives to the next period: <br>
$$L_{t+1}=n_tL_t+(1-\mu)L_t$$
Where $\mu$ is adult mortality. 

# Analytical solution using Sumpy

To study the fertility model, we need some variables that define income, consumption, population and more.

In [153]:
#Defining variables and parameters
Y = sm.symbols('Y_t')
alpha = sm.symbols('alpha')
L = sm.symbols('L_t')
A = sm.symbols('A')
X = sm.symbols('X')
n = sm.symbols('n_t')
lambda_ = sm.symbols('lambda')
tau = sm.symbols('tau')
beta= sm.symbols('beta')
mu = sm.symbols('mu')
c = sm.symbols('c_t')
y = sm.symbols('y_t')
u= sm.symbols('u_t')
L1 = sm.symbols('L_t+1')
e = sm.symbols('eta')
L_ss = sm.symbols('L^*')

Writing up the utility function

In [98]:
# Defining utility function
utility = beta*sm.ln(c)+(1-beta)*sm.ln(n)
utility

beta*log(c_t) + (1 - beta)*log(n_t)

The problem is to choose values for c and n such that the utility maximizes, given the following:

In [99]:
budgetconstraint = sm.Eq(c+lambda_*n,y*(1-tau))
budgetconstraint

Eq(c_t + lambda*n_t, y_t*(1 - tau))

Now solving with respect to c, which is the number of surviving children

In [100]:
c_solve = sm.solve(budgetconstraint, c)
c_substituted = c_solve[0]
c_substituted

-lambda*n_t - tau*y_t + y_t

Inserting this in the utility function to make it a function of n, we can maximize given the equation below

In [101]:
utility_substituted = utility.subs(c, c_substituted)
utility_substituted

beta*log(-lambda*n_t - tau*y_t + y_t) + (1 - beta)*log(n_t)

Taking the first order condition with respect to n to find the optimal level of births

In [130]:
first_order = sm.diff(utility_substituted, n)

first_order

-beta*lambda/(-lambda*n_t - tau*y_t + y_t) + (1 - beta)/n_t

The optimal number of births are then:

In [137]:
pop = sm.Eq(e, ((1-beta)/(lambda_))*(1-tau))
pop

Eq(eta, (1 - beta)*(1 - tau)/lambda)

In [103]:
n_solve = sm.solve(first_order, n)
n_solve[0]

y_t*(beta*tau - beta - tau + 1)/lambda

In [145]:
n_solvedd = sm.Eq(n, n_solve[0])
n_solvedd

Eq(n_t, y_t*(beta*tau - beta - tau + 1)/lambda)

In [152]:
sovledddd = n_solvedd.subs(e, (((1-beta)/(lambda_))*(1-tau)))
sovledddd

Eq(n_t, y_t*(beta*tau - beta - tau + 1)/lambda)

Going further, we look at the transition equation

In [104]:
transition = sm.Eq(n*L+(1-mu)*L, L1)
transition

Eq(L_t*n_t + L_t*(1 - mu), L_t+1)

Inserting per capita production equation into the fertility equation assuming:

In [180]:
income1 = sm.Eq(y, ((A*X)/L)**alpha)
income1

Eq(y_t, (A*X/L_t)**alpha)

In [172]:
transition_1 = transition.subs(n, n_solve[0])
transition_1

Eq(L_t*(1 - mu) + L_t*y_t*(beta*tau - beta - tau + 1)/lambda, L_t+1)

So, we get:

Using this term in the transition equation to get:

In [173]:
transition_2 = transition_1.subs(y, income1.rhs)
transition_2

Eq(L_t*(1 - mu) + L_t*(A*X)**alpha*(beta*tau - beta - tau + 1)/(L_t**alpha*lambda), L_t+1)

Now we assume steady state levels $$L_{t+1}=L_t=L^*$$

In [182]:
L_steadystate = sm.Eq(L, L_ss)
L_steadystate

Eq(L_t, L^*)

In [183]:
transition_3 = transition_2.subs(L, L_steadystate.rhs)
transition_4 = transition_3.subs(L1, L_steadystate.rhs)
transition_4

Eq(L^**(1 - mu) + L^**(A*X)**alpha*(beta*tau - beta - tau + 1)/(L^***alpha*lambda), L^*)

In this paper the steady state is taken for the population density, X. To get the steady state given the population density, we have to make the following derivations:
$$(L^*)^{1-\alpha}(AX)^\alpha\eta=L^*\mu$$
$$\Leftrightarrow L^*=(AX)\left({\frac{\eta}{\mu}}\right)^{\frac{1}{\alpha}}$$
Now, it is possible to divide by the area of land, so we can get the population density:
$$\Leftrightarrow \frac{L^*}{X}=A\left({\frac{\eta}{\mu}}\right)^{\frac{1}{\alpha}}$$
From this point on, whenever we refer to the steady state of population, we look at the expression above.
However, we must not forget to substitute $\eta\equiv \frac{(1-\beta)(1-\tau)}{\lambda}$

Solving the steady state level for population:

In [221]:
L_steadystate1 = sm.Eq((A*(((1-beta)*(1-tau))/(lambda_*mu))**(1/alpha)), L_ss)
L_steadystate1

Eq(A*((1 - beta)*(1 - tau)/(lambda*mu))**(1/alpha), L^*)

We have the steady state for population and can therefore insert it in the income equation to get the steady state income level $$y^*=\left(\frac{AX}{L^*}\right)^\alpha$$


In [192]:
income2 = income1.subs(L, L_steadystate1.lhs)
income2

Eq(y_t, (X/((1 - beta)*(1 - tau)/(lambda*mu))**(1/alpha))**alpha)

When Python reads the parameter values, it will return False if the exponential is negative. Therefore, we use the sm.Abs function to take the absolute values of the expression.

In [218]:
income3 = sm.Abs((((A*X)/L_steadystate1.lhs)**alpha))
income3

Abs((X/((1 - beta)*(1 - tau)/(lambda*mu))**(1/alpha))**alpha)

For the parameters to be used in a computational way, they need to be converted using the lambdify option from sympy.

In [219]:
steadystate_income = sm.lambdify((X, alpha, beta,lambda_,mu,tau), income3)

Assuming the following parameter values:
- $X = 1$
- $\alpha = 0.5$
- $\beta = 0.5$
- $\lambda = 1$
- $\mu = 0.01$
- $\tau = 0.4$

In [220]:
steadystate_income(1, 0.5, 0.5, 1, 0.01, 0.4)

0.03333333333333333